# Data pipeline 

## 1.1 Carga de Información

Ya que hemos configurado el manejador de base de datos ahora crearemos la DB, llamemosla `Transacciones`, como requerimos de las claves del service account para realizar la conexión no tenemos un gran riesgo de seguridad al dejar explícita la contraseña y el usser.  

In [ ]:
import psycopg2
import pandas as pd